In [77]:
import cv2
import numpy as np

In [78]:
# 定义一个自己读取图像的函数
def myimshow(arr):
    cv2.imshow(f'{arr.shape}',arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [79]:
# 加载图像数据并进行简单展示
image1 = cv2.imread('./01.png',cv2.IMREAD_ANYCOLOR)
# image1 = cv2.resize(image1,(448,448))
myimshow(image1)

In [80]:
# 转换为灰度图
image2 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
# 并且使用阈值将可能是毛孔的低像素值区域与其它区域进行二值化
# 
t,thresMat = cv2.threshold(image2,120,255,cv2.THRESH_BINARY_INV)
total_1 = np.concatenate((image2,thresMat),axis=-1)
myimshow(total_1)

In [81]:
# 形态学膨胀操作将细微的毛孔高亮区域进行放大
kernel1 = np.ones((3,3),np.uint8)
thresMat1 = cv2.dilate(thresMat,kernel=kernel1,iterations=3)
myimshow(thresMat1)

In [82]:
# 获取模板轮廓(mode=只检测外轮廓，method=只保留终点坐标)
contours, hierarchy = cv2.findContours(thresMat1, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
# 框出选择出来的区域
image3 = cv2.drawContours(image2.copy(), contours, contourIdx=-1, color=(0, 0, 255), thickness=2)
# 进行图像拼接
total_2 = np.concatenate((image2,thresMat1,image3),axis=-1)
myimshow(total_2)

In [83]:
# 获取模板所有轮廓的外接矩形
areas = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    ar = w / float(h)
    if ar < 2 and (0 < w < 20) and (0 < h < 20):
        areas.append((x, y, w, h))

# cv2.rectangle(card, (x-5, y-5), (x + w+5, y + h+5), color=(0, 0, 255), thickness=2)
# cv2.putText(card, "".join(groups), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

In [84]:
image1_copy  = image1.copy()
for idx,(x,y,w,h)in enumerate(areas):
    cv2.rectangle(image1_copy, (x-2, y-2), (x + w+2, y + h+2), color=(0, 0, 255), thickness=1)
    # cv2.putText(image1_copy, 'pore', (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
myimshow(image1_copy)

In [85]:
cv2.imwrite('./pore.jpg',image1_copy)

True